In [1]:
# Importing necessary libraries

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Load the dataset
data = pd.read_csv('house_prices.csv')
print("First 5 rows of the dataset:")
print(data.head())

# Dataset Info
print("\nDataset Information:")
print(data.info())

# Checking for missing values
print("\nMissing Values:")
print(data.isnull().sum())

# Fill missing values 
data['Size'].fillna(data['Size'].median(), inplace=True)
data['Number of Rooms'].fillna(data['Number of Rooms'].median(), inplace=True)

# Handling outliers
upper_limit = data['Price'].quantile(0.95)
data['Price'] = np.where(data['Price']> upper_limit, upper_limit, data['Price'])

# Encoding the 'Location' using 